## Overview
This script takes the *clean* version of the inventory report from Wilco and the static details about the SKU. It then creates a master list that has both the pieces of information.

In [30]:
import re
import os
import openpyxl
from pathlib import Path

IN_DIR = 'sample1'
OUT_DIR = Path(IN_DIR, 'out')

if not os.path.exists(OUT_DIR):
    raise ValueError("Output directory not found")


In [31]:
IN_DETAILS = Path(IN_DIR, 'lb_inventory_meta.xlsx')
IN_STOCK = Path(OUT_DIR, 'wilco_2022-08-05.xlsx')

m = re.search(r'wilco_(.*).xlsx', str(IN_STOCK))
if m:
    out_fname = "master_" + m.group(1) + ".xlsx"
else:
    raise ValueError(f"Wilco inventory file '{IN_STOCK}' has wrong naming conventions")
    
OUT_MASTER = Path(OUT_DIR, out_fname)

print(f"Merging '{IN_DETAILS}' with '{IN_STOCK}' to create '{OUT_MASTER}'")

Merging 'sample1/lb_inventory_meta.xlsx' with 'sample1/out/wilco_2022-08-05.xlsx' to create 'sample1/out/master_2022-08-05.xlsx'


In [32]:
wb = openpyxl.load_workbook(IN_DETAILS)
print(wb)

In [33]:
sheet = wb['Inventory']

In [34]:
if sheet[1][0].value == "Location" and sheet[1][1].value == "SKU" and sheet[1][2].value == "Type":
    print('Matching column names found')

In [35]:
costs = {}
for row in sheet.iter_rows(min_row=2, values_only=True):
    if row[1] == None:
        break
    sku = row[1].replace('\n', '')
    sku_details = {
        'Type'       : row[3],
        'Description': row[2],
        'PcsPerCase' : row[5],
        'WtPerPcGms' : row[6],
        'PricePerPc' : row[7]
    }
    costs[sku] = sku_details

In [36]:
wb = openpyxl.load_workbook(IN_STOCK)
sheet = wb.active
print(sheet)

<Worksheet "Sheet">


In [37]:
items = {}
for row in sheet.iter_rows(min_row=2, values_only=True):
    sku = row[0].replace('\n', '')
    sku_details = {
        'OnHand' : row[2]
    }
    items[sku] = sku_details

In [38]:
inventory_items = []
for sku in items.keys():
    item = costs[sku]
    item['PcPerCase'] = costs[sku]['PcsPerCase']
    item['PcWtGms'] = costs[sku]['WtPerPcGms']
    item['PcPrice'] = costs[sku]['PricePerPc']
    if item['PcPrice'] == None:
        case_cost = 0
        total_cost = 0
    else:
        case_cost = float(item['PcPrice']) * int(item['PcPerCase'])
        total_cost = items[sku]['OnHand'] * case_cost
        
    if item['PcWtGms'] == None:
        total_wt_lbs = 0
    else:
        total_wt_lbs = (float(item['PcWtGms']) * int(item['PcPerCase']) * int(items[sku]['OnHand'])) * 0.0022
        
    inventory_items.append((sku, item['Type'], item['Description'], item['PcPerCase'], item['PcWtGms'], 
                            item['PcPrice'], case_cost, items[sku]['OnHand'], total_cost, total_wt_lbs))

In [39]:
print(inventory_items)

[('001', 'Frozen', 'Malabar Parotta 28/ctn', 13, 1, 2, 26.0, 58, 1508.0, 1.6588), ('003-20pk', 'Frozen', 'Tapioca Sliced 400', 14, 2, 3, 42.0, 4, 168.0, 0.2464), ('003-25pk', 'Frozen', 'Tapioca Sliced', 15, 3, 4, 60.0, 66, 3960.0, 6.534000000000001), ('004', 'Frozen', 'Hand Grated Coconut', 16, 4, 5, 80.0, 178, 14240.0, 25.0624), ('010', 'Frozen', 'Sliced Coconut', 17, 5, 6, 102.0, 157, 16014.0, 29.359), ('022', 'Dry', 'Mixed Vegetable Pickle', 18, 6, 7, 126.0, 2, 252.0, 0.4752), ('023', 'Dry', 'White Lemon Pickle', 19, 7, 8, 152.0, 2, 304.0, 0.5852), ('026', 'Dry', 'Green Chilli Pickle', 20, 8, 9, 180.0, 1, 180.0, 0.35200000000000004), ('027', 'Dry', 'Ginger Bitter Lemon (Vadukapuli) Red Pickle', 21, 9, 10, 210.0, 2, 420.0, 0.8316), ('028', 'Dry', 'Giant Bitter Lemon (Vadukapuli) White Pickle', 22, 10, 11, 242.0, 1, 242.0, 0.48400000000000004), ('029', 'Dry', 'Garlic Pickle', 23, 11, 12, 276.0, 3, 828.0, 1.6698000000000002), ('036', 'Dry', 'Bitterground Pickle', 24, 12, 13, 312.0, 3, 

In [40]:
out_wb = openpyxl.Workbook()
out_ws = out_wb.active
out_ws.append(['SKU', 'Type', 'Description', 'PcPerCase', 'PcWtGms', 'PcCost', 'CaseCost', 'OnHand', 'TotalCost', 'TotalWtLbs'])

In [41]:
for row in inventory_items:
    out_ws.append(row)

In [42]:
out_wb.save(OUT_MASTER)